In [ ]:
from dotenv import load_dotenv
import os
import psycopg
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai = OpenAI()

dbuser = os.getenv("PGUSER")
dbpassword = os.getenv("PGPASSWORD")
dbhost = os.getenv("PGHOST")
dbname = os.getenv("PGDATABASE")


In [ ]:
with psycopg.connect(f"dbname={dbname} user={dbuser} host={dbhost} password={dbpassword}") as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM books LIMIT 10")
        print(cur.fetchall())

In [ ]:

def get_books():
    with psycopg.connect(f"dbname={dbname} user={dbuser} host={dbhost} password={dbpassword}") as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT (title, author, genre, datecompleted, datestartedreading, shortstory) FROM books")
            return cur.fetchall()

In [ ]:
def create_ai_prompt_from_books(books_data: list[tuple]) -> str:
    """
    Transforms a list of book data tuples into a well-structured, human-readable
    string suitable for an AI model's context.

    Args:
        books_data: A list of tuples, where each tuple contains book information
                    in the order: (title, author, genre, datecompleted, datestartedreading, shortstory).

    Returns:
        A formatted string containing all the book data, with each book
        separated by a clear delimiter.
    """
    # A list to hold the formatted strings for each book
    formatted_books = []

    # Iterate through the list of book tuples
    for book_outer_tuple in books_data:
        book = book_outer_tuple[0]

        # Unpack the tuple for easier access to each field
        title, author, genre, datecompleted, datestartedreading, shortstory = book

        # Format the information for a single book.
        # We use a f-string for easy readability and variable insertion.
        # The formatting is designed to be clear and consistent for the AI.
        book_string = (
            f"Title: {title}\n"
            f"Author: {author}\n"
            f"Genre: {genre}\n"
            f"Date Started Reading: {datestartedreading}\n"
            f"Date Completed: {datecompleted}\n"
            f"Is a Short Story: {'No' if shortstory == 'f' else 'Yes'}\n"
        )
        formatted_books.append(book_string)

    # Join all the formatted book strings together with a separator
    # to make each book a distinct block of information.
    # The final string is then returned.
    return "\n" + "---" * 15 + "\n".join(formatted_books)

In [ ]:
# --- Example Usage ---
# 1. First, you would call your get_books() function to fetch the data.
# For this example, we'll use the mock data function.
books_from_db = get_books()

# 2. Then, pass the fetched data to the function to create the prompt string.
ai_context_prompt = create_ai_prompt_from_books(books_from_db)

# 3. Print the final prompt to see the result.
# This is the string you would then pass to your AI model.
# print(ai_context_prompt)

In [ ]:
system_prompt = """
    You are a helpful assistant that can answer questions about books.
    You are given a list of books and their details.
    You can use this information to answer questions about the books.
    You can also use this information to answer questions about the user's reading habits.
    You can also use this information to answer questions about the user's reading preferences.
"""

system_prompt += f"With this context, please chat with the user, always staying in character."

system_prompt += f"\n{ai_context_prompt}"

def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content
    

In [ ]:
gr.ChatInterface(chat, type="messages").launch()